In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

In [3]:
from hdna import * 
import nupack as nu

In [4]:
model = Model('dna', '3D', stacking='nostacking')
A = Strand(model, "AAAAAAAAA")
B = A.complementary()
C = Chamber(model, A, B)
N = Kinetwork(model, A, B, Geometry(120,270), clean=True)

In [5]:
for s in C.slidings:
    print(s.structure)
    for bf in s.backfray:
        print(bf.structure)
    print('\n')

(........+)........


((.......+)).......
(........+.).......
.(.......+)........


(((......+)))......
(........+..)......
.(.......+.).......
..(......+)........


((((.....+)))).....
(........+...).....
.(.......+..)......
..(......+.).......
...(.....+)........


(((((....+)))))....
(........+....)....
.(.......+...).....
..(......+..)......
...(.....+.).......
....(....+)........


((((((...+))))))...
(........+.....)...
.(.......+....)....
..(......+...).....
...(.....+..)......
....(....+.).......
.....(...+)........


(((((((..+)))))))..
(........+......)..
.(.......+.....)...
..(......+....)....
...(.....+...).....
....(....+..)......
.....(...+.).......
......(..+)........


((((((((.+)))))))).
(........+.......).
.(.......+......)..
..(......+.....)...
...(.....+....)....
....(....+...).....
.....(...+..)......
......(..+.).......
.......(.+)........


.((((((((+.))))))))
.(.......+........)
..(......+.......).
...(.....+......)..
....(....+.....)...
.....(...+....)....
....

In [6]:
import networkx as nx 

def getix(string):
    lix = []
    rix = []
    for i, e in enumerate(list(string)):
        if e == '(':
            lix.append(i)
        if e == ')':
            rix.append(i)
    return [(l,r) for l,r in zip(lix, rix[::-1])]

def adjacent(source, target):
    adj = []
    tx = getix(target)
    sx = getix(source)
    for i, item in enumerate(tx):
        if len(sx) > 1:
            check = [sx[0], sx[-1]]
            if item in check:
                print('item',item)
                print('check',check)
                if i == 0:
                    print('zero')
                    # adj.append(tx[i+1])
                elif i == len(tx)-1:
                    print('last')
                    # adj.append(tx[i-1])
                else:
                    if tx[i-1] not in sx:
                        if tx[i-1] not in adj:
                            adj.append(tx[i-1])
                    if tx[i+1] not in sx:
                        if tx[i+1] not in adj:
                            adj.append(tx[i+1])
        else:
            check = sx
            if item in check:
                if i == 0:

                    adj.append(tx[i+1])
                elif i == len(tx)-1:
                    adj.append(tx[i-1])
                else:
                    adj.append(tx[i-1])
                    adj.append(tx[i+1])
    return adj 

def update_structure(structure, target):
    updated = structure
    moveslist = []
    moves = adjacent(structure, target)
    print(moves)
    for m in moves:
        up = updated[:m[0]] + '(' + updated[m[0]+1:m[1]] + ')' + updated[m[1]+1:]
        moveslist.append(up)
    return moveslist

def leafs(structure, target, graph):
    moves = update_structure(structure, target)
    for move in moves:
        print(structure, move)
        graph.add_edge(structure, move)
        yield list(leafs(move, target, graph))

print('\n')


In [7]:
s1 = C.slidings[6]
s2 = C.slidings[2]
print(s.structure)

fs1 = s1.structure
bf11 = s1.backfray[3].structure
bf12 = s1.backfray[0].structure
full1 = getix(fs1)
bfix1 = getix(bf11)
bfix2 = getix(bf12)

G11 = nx.DiGraph()
G12 = nx.DiGraph()
list(leafs(bf11, fs1, G11))
list(leafs(bf12, fs1, G12))

G = nx.compose(G11, G12)

........(+........)
[(2, 14), (4, 12)]
...(.....+...)..... ..((.....+...))....
item (2, 14)
check [(2, 14), (3, 13)]
item (3, 13)
check [(2, 14), (3, 13)]
[(1, 15), (4, 12)]
..((.....+...)).... .(((.....+...)))...
item (1, 15)
check [(1, 15), (3, 13)]
item (3, 13)
check [(1, 15), (3, 13)]
[(0, 16), (4, 12)]
.(((.....+...)))... ((((.....+...))))..
item (0, 16)
check [(0, 16), (3, 13)]
zero
item (3, 13)
check [(0, 16), (3, 13)]
[(4, 12)]
((((.....+...)))).. (((((....+..)))))..
item (0, 16)
check [(0, 16), (4, 12)]
zero
item (4, 12)
check [(0, 16), (4, 12)]
[(5, 11)]
(((((....+..))))).. ((((((...+.))))))..
item (0, 16)
check [(0, 16), (5, 11)]
zero
item (5, 11)
check [(0, 16), (5, 11)]
[(6, 10)]
((((((...+.)))))).. (((((((..+)))))))..
item (0, 16)
check [(0, 16), (6, 10)]
zero
item (6, 10)
check [(0, 16), (6, 10)]
last
[]
.(((.....+...)))... .((((....+..))))...
item (1, 15)
check [(1, 15), (4, 12)]
item (4, 12)
check [(1, 15), (4, 12)]
[(0, 16), (5, 11)]
.((((....+..))))... (((((....+..))

nx.write_gexf(G, './checksliding.gexf')

In [8]:
N.zippingraph()

In [9]:
N.slidingraph()

In [10]:
from itertools import combinations

In [15]:
def filternodes(property, function, graph):
    def filternode(node):
        try: 
            try: 
                return function(graph.nodes[node][property])
            except KeyError: pass
        except TypeError: 
            try: 
                value = function([e[1][property] for e in list(N.SG.nodes.data())])
            except KeyError: pass 
            return graph.nodes[node][property] == value
    Rgraph = nx.subgraph_view(graph, filter_node=filternode)
    return Rgraph

left = filternodes('side', lambda x: x == 'l', N.SG)
right = filternodes('side', lambda x: x == 'r', N.SG)
# print(*list(left.nodes.data()), sep='\n')
# print()
# print(*list(right.nodes.data()), sep='\n')
lcompl = filternodes('dtc', lambda x: x == 0, left)
rcompl = filternodes('dtc', lambda x: x == 0, right)
# print(*list(lcompl.nodes.data()), sep='\n')
# print(*list(rcompl.nodes.data()), sep='\n')

for l, r in combinations([*list(lcompl.nodes.data()), *list(rcompl.nodes.data())], 2):
    print(l[1]['side'],'                 ', r[1]['side'])
    print(l[1]['dpxdist'],'                 ', r[1]['dpxdist'])
    print(l[0], r[0])
    if l[1]['side'] == r[1]['side']:
        distance = abs(l[1]['dpxdist'] - r[1]['dpxdist'])
        print('sliding distance:',distance)
    else:
        distance = abs(l[1]['dpxdist'] + r[1]['dpxdist'])
        print('sliding distance:',distance)

# SELECT COMPLETED SLIDINGS
completed = filternodes('dtc', lambda x: x == 0, N.SG)
nucleations = filternodes('pairs', lambda x: x == 1, N.SG)


# print(*list(completed.nodes.data()), sep='\n')
# print(*list(nucleations.nodes.data()), sep='\n')
# print(completed.nodes())
# for node in completed.nodes():
#     sldnext = filternodes('dpxdist', lambda x: 3 < x < 3 + completed.nodes[node]['dpxdist'], completed)
#     print(node, 'next:')
#     print(*list(sldnext.nodes()), sep='\n')

# for n1, n2 in combinations(list(completed.nodes()), 2):
#     print(n1, n2)
#     # if abs(n1.dpxdist - n2.dpxdist) < 2:
#     #     N.SG.add_edge(n1, n2)

l                   l
7                   6
((.......+))....... (((......+)))......
sliding distance: 1
l                   l
7                   5
((.......+))....... ((((.....+)))).....
sliding distance: 2
l                   l
7                   4
((.......+))....... (((((....+)))))....
sliding distance: 3
l                   l
7                   3
((.......+))....... ((((((...+))))))...
sliding distance: 4
l                   l
7                   2
((.......+))....... (((((((..+)))))))..
sliding distance: 5
l                   l
7                   1
((.......+))....... ((((((((.+)))))))).
sliding distance: 6
l                   r
7                   7
((.......+))....... .......((+.......))
sliding distance: 14
l                   r
7                   6
((.......+))....... ......(((+......)))
sliding distance: 13
l                   r
7                   5
((.......+))....... .....((((+.....))))
sliding distance: 12
l                   r
7                   4
((.......+)).....

In [17]:
nucleations = filternodes('pairs', lambda x: x == 1, N.SG)
print(*list(nucleations.nodes()), sep='\n')

(........+.).......
.(.......+)........
.......(.+........)
........(+.......).
(........+..)......
.(.......+.).......
..(......+)........
......(..+........)
.......(.+.......).
........(+......)..
(........+...).....
.(.......+..)......
..(......+.).......
...(.....+)........
.....(...+........)
......(..+.......).
.......(.+......)..
........(+.....)...
(........+....)....
.(.......+...).....
..(......+..)......
...(.....+.).......
....(....+)........
....(....+........)
.....(...+.......).
......(..+......)..
.......(.+.....)...
........(+....)....
(........+.....)...
.(.......+....)....
..(......+...).....
...(.....+..)......
....(....+.).......
.....(...+)........
...(.....+........)
....(....+.......).
.....(...+......)..
......(..+.....)...
.......(.+....)....
........(+...).....
(........+......)..
.(.......+.....)...
..(......+....)....
...(.....+...).....
....(....+..)......
.....(...+.).......
......(..+)........
..(......+........)
...(.....+.......).
....(....+......)..


In [18]:
for n in N.ZG.nodes.data():
    try: n[1]['obj'] = str(type(n[1]['obj']))   
    except: pass 
nx.write_gexf(N.ZG, './completeZG.gexf')

In [45]:
print(*list(N.ZG.nodes.data()), sep='\n')

('(........+........)', {'obj': "<class 'hdna.complex.Zippo'>", 'pairs': 1})
('((.......+.......))', {'obj': "<class 'hdna.complex.Zippo'>", 'pairs': 2})
('(((......+......)))', {'obj': "<class 'hdna.complex.Zippo'>", 'pairs': 3})
('((((.....+.....))))', {'obj': "<class 'hdna.complex.Zippo'>", 'pairs': 4})
('(((((....+....)))))', {'obj': "<class 'hdna.complex.Zippo'>", 'pairs': 5})
('((((((...+...))))))', {'obj': "<class 'hdna.complex.Zippo'>", 'pairs': 6})
('(((((((..+..)))))))', {'obj': "<class 'hdna.complex.Zippo'>", 'pairs': 7})
('((((((((.+.))))))))', {'obj': "<class 'hdna.complex.Zippo'>", 'pairs': 8})
('(((((((((+)))))))))', {'obj': "<class 'hdna.complex.Zippo'>", 'pairs': 9})
('.(.......+.......).', {'obj': "<class 'hdna.complex.Zippo'>", 'pairs': 1})
('.((......+......)).', {'obj': "<class 'hdna.complex.Zippo'>", 'pairs': 2})
('.(((.....+.....))).', {'obj': "<class 'hdna.complex.Zippo'>", 'pairs': 3})
('.((((....+....)))).', {'obj': "<class 'hdna.complex.Zippo'>", 'pairs': 4})

In [50]:
list(nx.compose(N.SG, N.ZG).nodes())

['.........+.........',
 '(((((((((+)))))))))',
 '(........+.).......',
 '((.......+)).......',
 '.(.......+)........',
 '.......(.+........)',
 '.......((+.......))',
 '........(+.......).',
 '(........+..)......',
 '((.......+.))......',
 '(((......+)))......',
 '.(.......+.).......',
 '.((......+)).......',
 '..(......+)........',
 '......(..+........)',
 '......((.+.......))',
 '......(((+......)))',
 '.......(.+.......).',
 '.......((+......)).',
 '........(+......)..',
 '(........+...).....',
 '((.......+..)).....',
 '(((......+.))).....',
 '((((.....+)))).....',
 '.(.......+..)......',
 '.((......+.))......',
 '.(((.....+)))......',
 '..(......+.).......',
 '..((.....+)).......',
 '...(.....+)........',
 '.....(...+........)',
 '.....((..+.......))',
 '.....(((.+......)))',
 '.....((((+.....))))',
 '......(..+.......).',
 '......((.+......)).',
 '......(((+.....))).',
 '.......(.+......)..',
 '.......((+.....))..',
 '........(+.....)...',
 '(........+....)....',
 '((.......+...)

In [51]:
N.completegraph()

In [54]:
print(N.DG)
print(N.SG)
print(N.ZG)

DiGraph with 284 nodes and 984 edges
DiGraph with 240 nodes and 840 edges
DiGraph with 45 nodes and 144 edges


In [55]:
for n in N.DG.nodes.data():
    try: n[1]['obj'] = str(type(n[1]['obj']))   
    except: pass 
nx.write_gexf(N.DG, './COMPLETE.gexf')